# Phylogenetics Tutorial: From Sequences to Trees

**Created for [Evomics](https://learn.evomics.org)**

This tutorial will guide you through a complete phylogenetic analysis workflow:

1. **Loading and preparing sequence data**
2. **Sequence alignment** (using MAFFT)
3. **Model selection** (using ModelFinder)
4. **Tree inference** (using IQ-TREE)
5. **Tree visualization** (using toytree)
6. **Interpretation exercises** (bootstrap values, branch lengths)

---

## Learning Objectives

By the end of this tutorial, you will be able to:
- Understand the basics of phylogenetic inference
- Select appropriate evolutionary models for your data
- Construct maximum likelihood phylogenetic trees
- Interpret bootstrap support values and branch lengths
- Visualize and annotate phylogenetic trees

---

## 1. Environment Setup

First, we'll install the necessary bioinformatics tools and Python packages.

In [ ]:
# Install bioinformatics tools
import os
import subprocess

print("📦 Installing bioinformatics tools...")

# Install MAFFT (available in apt)
print("  → Installing MAFFT...")
!apt-get update -qq 2>&1 > /dev/null
!apt-get install -y -qq mafft 2>&1 > /dev/null

# Install IQ-TREE (download pre-compiled binary)
if not os.path.exists('/usr/local/bin/iqtree'):
    print("  → Installing IQ-TREE...")
    !wget -q http://github.com/iqtree/iqtree2/releases/download/v2.3.6/iqtree-2.3.6-Linux-intel.tar.gz
    !tar -xzf iqtree-2.3.6-Linux-intel.tar.gz
    !cp iqtree-2.3.6-Linux-intel/bin/iqtree2 /usr/local/bin/iqtree
    !chmod +x /usr/local/bin/iqtree
    !rm -rf iqtree-2.3.6-Linux-intel*
else:
    print("  → IQ-TREE already installed")

# Install Python packages
print("  → Installing Python packages...")
!pip install -q biopython toytree toyplot pandas matplotlib seaborn

# Verify installations
print("\n✅ Installation complete! Verifying tools:")
result = subprocess.run(['mafft', '--version'], capture_output=True, text=True)
if result.returncode == 0:
    print(f"   MAFFT: {result.stderr.split()[0]}")
    
result = subprocess.run(['iqtree', '--version'], capture_output=True, text=True)
if result.returncode == 0:
    version_line = result.stdout.split('\n')[0]
    print(f"   IQ-TREE: {version_line.split()[2]}")

print("\n🎉 Ready to start!")

In [ ]:
# Import required libraries
import os
import subprocess
from Bio import SeqIO, AlignIO, Phylo
import toytree
import toyplot
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import io

# Detect if running in Colab
try:
    from google.colab import files
    IN_COLAB = True
except ImportError:
    IN_COLAB = False
    print("📍 Running locally (not in Colab)")

# Set style for plots
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 8)

print("✓ All libraries loaded successfully!")

## 2. Data Preparation

We'll use a sample dataset of mammalian cytochrome b sequences for demonstration.

### Example Dataset

We'll use a sample dataset of mammalian cytochrome b sequences for demonstration.

In [ ]:
# Create example dataset: Mammalian cytochrome b sequences
example_sequences = """>Human
ATGACCAATATTCGAAAATCACACCCCCTAATAAAAATTATTAACAACTCATTCATTGACCTACCAGCAC
CATCAAACATTTCATCATGATGAAACTTTGGCTCCCTCCTAGGAATCTGCCTAATTCTACAAATCCTAAC
AGGCCTATTTCTAGCAATACACTACACATCAGACACAATAACAGCATTTTCATCAGTCACCCACATCTGC
CGAGACGTAAATTATGGCTGAATTATCCGATATATACATGCAAACGGAGCATCCATATTCTTTATCTGCC
TATTCCTACACGTAGGACGAGGCCTATATTACGGATCATTTCTCTACATAAAAGAGTGTGAAACATCGGA
GTAATTCTCCTACTCACAGTAATAGCCACAGCATTCATAGGCTATGTCCTACCATGAGGACAAATATCAT
TTTGAGGGGCAACAGTAATTACAAACTTACTATCCGCCATCCCATACATTGGGACAGACCTAGTACAATG
AATCTGAGGGGGATTCTCAGTAGACAAAGCAACCCTTACACGATTCTTTGCATTTCACTTTATCCTTCCA
TTTATTATTGCAGCCCTCGCCATAGTCCACCTCCTCTTCCTCCACGAAACGGGATCCAACAACCCAACAG
GACTAACATCCGACTCAGACAAAATCACCTTTCACCCCTACTACACAATCAAAGACATCCTAGGCGCCCT
>Chimpanzee
ATGACCAACATTCGAAAATCACACCCCCTAATAAAAATTATTAACAACTCATTCATTGACCTACCAGCAC
CATCAAACATTTCATCATGATGAAACTTTGGCTCCCTCCTAGGAATTTGCCTAATTCTACAAATCCTAAC
AGGCCTATTTCTAGCAATACACTACACATCAGACACAATAACAGCATTTTCATCAGTCACCCACATCTGC
CGAGACGTAAATTATGGCTGAATTATCCGATATATACATGCAAACGGAGCATCCATATTCTTTATCTGCC
TATTCCTACACGTAGGACGAGGCCTATATTACGGATCATTTCTCTACATAAAAGAGTGTGAAACATCGGA
GTAATTCTCCTACTCACAGTAATAGCCACAGCATTCATAGGCTATGTCCTACCATGAGGACAAATATCAT
TTTGAGGGGCAACAGTAATTACAAACTTACTATCCGCCATCCCATACATTGGGACAGACCTAGTACAATG
AATCTGAGGGGGATTCTCAGTAGACAAAGCAACCCTTACACGATTCTTTGCATTTCACTTTATCCTTCCA
TTTATTATTGCAGCCCTCGCCATAGTCCACCTCCTCTTCCTCCACGAAACGGGATCCAACAACCCAACAG
GACTAACATCCGACTCAGACAAAATCACCTTTCACCCCTACTACACAATCAAAGACATCCTAGGCGCCCT
>Gorilla
ATGACCAACATTCGAAAATCACACCCCCTAATAAAAATTATTAACAACTCATTCATTGACCTACCAGCAC
CATCAAACATTTCATCATGATGAAACTTTGGCTCCCTCCTAGGAATTTGCCTAATTCTACAAATCCTAAC
AGGCCTATTTCTAGCAATACACTACACATCAGACACAATAACAGCATTTTCATCAGTCACCCACATCTGC
CGAGACGTAAATTATGGCTGAATTATCCGATATATACATGCAAACGGAGCATCCATATTCTTTATCTGCC
TATTCCTACACGTAGGACGAGGCCTATATTACGGATCATTTCTCTACATAAAAGAGTGTGAAACATCGGA
GTAATTCTCCTACTCACAGTAATAGCCACAGCATTCATAGGCTATGTCCTACCATGAGGACAAATATCAT
TTTGAGGGGCAACAGTAATTACAAACTTACTATCCGCCATCCCATACATTGGGACAGACCTAGTACAATG
AATCTGAGGGGGATTCTCAGTAGACAAAGCAACCCTTACACGATTCTTTGCATTTCACTTTATCCTTCCA
TTTATTATTGCAGCCCTCGCCATAGTCCACCTCCTCTTCCTCCACGAAACGGGATCCAACAACCCAACAG
GACTAACATCCGACTCAGACAAAATCACCTTTCACCCCTACTACACAATCAAAGACATCCTAGGCGCCCT
>Mouse
ATGACCAATATAAGAAAGACACACTTCTAATAAAAATCATCAACAGCTCATTCATTGACCTTCCAACACC
ATCCAACATCTCGTCATGATGAAACTTTGGTTCCCTCTTAGGAATATGCTTAATTACCCAAATCTTAACA
GGCCTATTCCTAGCCATACACTACACATCAGACACCACAACAGCCTTCTCCTCAGTCACTCATATCTGCC
GAGACGTAAACTACGGATGACTAATCCGCAACATACACGCCAACGGAGCATCCTTCTTCTTCATCTGCCT
ATTCCTACACGTAGGCCGAGGCCTGTACTATGGCTCATACCTTTACAAAGAAACATGAAACATTGGAGTA
GTCCTCCTACTAATAGTTATAGCCACAGCCTTCGTAGGCTATGTCCTACCATGAGGACAAATATCATTTT
GAGGAGCCACAGTAATTACAAACCTACTATCAGCAGTCCCCTACATAGGAGACTCCCTAGTACAATGAAT
CTGAGGTGGATTCTCAGTAGACAACGCAACTCTCACCCGATTCTTTGCATTCCACTTCCTCCTCCCATTC
ATCATTGTAGCAGTAGTCATAGTACACCTACTCTTCCTCCACGAAACAGGGTCCAATAACCCCTCAGGAC
TAATATCCAATTCTGACAAAATTCAATTCCACCCCTACTACTCCACCAAAGACATTCTAGGAGCCCTATT
>Rat
ATGACCAATATAAGAAAGACACACTTCTAATAAAAATCATCAACAGCTCATTCATTGACCTTCCAACACC
ATCCAACATCTCGTCATGATGAAACTTTGGTTCCCTCTTAGGAATATGCTTAATTACCCAAATCTTAACA
GGCCTATTCCTAGCCATACACTACACATCAGACACCACAACAGCCTTCTCCTCAGTCACTCATATCTGCC
GAGACGTAAACTACGGATGACTAATCCGCAACATACACGCCAACGGAGCATCCTTCTTCTTCATCTGCCT
ATTCCTACACGTAGGCCGAGGCCTGTACTATGGCTCATACCTTTACAAAGAAACATGAAACATTGGAGTA
GTCCTCCTACTAATAGTTATAGCCACAGCCTTCGTAGGCTATGTCCTACCATGAGGACAAATATCATTTT
GAGGAGCCACAGTAATTACAAACCTACTATCAGCAGTCCCCTACATAGGAGACTCCCTAGTACAATGAAT
CTGAGGTGGATTCTCAGTAGACAACGCAACTCTCACCCGATTCTTTGCATTCCACTTCCTCCTCCCATTC
ATCATTGTAGCAGTAGTCATAGTACACCTACTCTTCCTCCACGAAACAGGGTCCAATAACCCCTCAGGAC
TAATATCCAATTCTGACAAAATTCAATTCCACCCCTACTACTCCACCAAAGACATTCTAGGAGCCCTATT
>Whale
ATGACGAACATCCGAAAAACCCACCCCTTAATAAAAATCGTAAACAGCTCACTAGTTGATCTGCCCGCCC
CATCCAACATCTCAGCCTGATGAAACTTCGGCTCATTACTAGGAATCTGCCTAATAACACAAATTTTAAC
AGGACTATTCCTAGCCATACACTACACCTCAGATATCTCAACAGCCTTCTCATCAGTCACACATATCTGC
AGAGACGTAAACTACGGCTGAATTATCCGATTTATACACGCCAACGGAGCATCCTTCTTCTTCATCTGTA
TCTACCTACACATCGCCCGAGGACTATACTACGGCTCATATCTCTACAAAGAAACATGAAACATCGGAGT
AATCCTCCTCCTCACAGTAATAGCCACAGCATTCATAGGTTACGTCCTACCATGAGGACAAATATCATTT
TGAGGGGCCACAGTAATTACAAACCTTCTATCCGCTATCCCATACATTGGAAACACCCTCGTCCAATGAG
TCTGAGGTGGATTCTCAGTAGACAACGCAACACTTACACGATTCTTCGCATTCCACTTTATCCTACCATT
TATTATTGCAGCTCTAACCATAGTCCACTTACTATTTCTCCACGAAACAGGATCCAACAACCCCTCAGGA
TTAAACTCAGACTCAGACAAAATCCCCTTCCACCCATACTTCTCATACAAAGACATCCTAGGATTTACAC
"""

# Save example sequences
with open('example_sequences.fasta', 'w') as f:
    f.write(example_sequences)

input_file = 'example_sequences.fasta'

# Read and display sequence information
sequences = list(SeqIO.parse(input_file, 'fasta'))
print(f"\n📊 Dataset Information:")
print(f"   Number of sequences: {len(sequences)}")
print(f"   Sequence names: {[seq.id for seq in sequences]}")
print(f"   Sequence lengths: {[len(seq.seq) for seq in sequences]}")
print(f"\n✓ Data loaded successfully!")

## 3. Multiple Sequence Alignment with MAFFT

MAFFT is a fast and accurate multiple sequence alignment program. We'll use it to align our sequences if they aren't already aligned.

### Key MAFFT Parameters:
- `--auto`: Automatically selects the appropriate strategy
- `--thread -1`: Uses all available CPU threads
- `--quiet`: Reduces output verbosity

In [ ]:
# Run MAFFT alignment
aligned_file = 'aligned_sequences.fasta'

print("🔄 Running MAFFT alignment...")
mafft_cmd = f"mafft --auto --quiet {input_file} > {aligned_file}"
result = subprocess.run(mafft_cmd, shell=True, capture_output=True, text=True)

if result.returncode == 0:
    print("✓ Alignment completed successfully!")
    
    # Display alignment statistics
    alignment = AlignIO.read(aligned_file, 'fasta')
    print(f"\n📊 Alignment Statistics:")
    print(f"   Number of sequences: {len(alignment)}")
    print(f"   Alignment length: {alignment.get_alignment_length()}")
    
    # Calculate consensus sequence (simple majority rule)
    from collections import Counter
    alignment_length = alignment.get_alignment_length()
    consensus = []
    
    for i in range(alignment_length):
        column = alignment[:, i]
        # Count characters (excluding gaps for consensus)
        chars = [c.upper() for c in column if c != '-']
        if chars:
            # Get most common character
            most_common = Counter(chars).most_common(1)[0][0]
            consensus.append(most_common)
        else:
            consensus.append('-')
    
    consensus_seq = ''.join(consensus)
    print(f"   Consensus sequence (first 50 bp): {consensus_seq[:50]}...")
    
    # Calculate percent identity to consensus
    identities = []
    for record in alignment:
        matches = sum(1 for i, base in enumerate(str(record.seq)) 
                     if base.upper() == consensus[i] and base != '-')
        non_gap_positions = sum(1 for base in str(record.seq) if base != '-')
        if non_gap_positions > 0:
            identities.append(matches / non_gap_positions * 100)
    
    print(f"   Mean identity to consensus: {sum(identities)/len(identities):.1f}%")
else:
    print(f"❌ Error running MAFFT: {result.stderr}")

### Visualize Alignment Quality

In [ ]:
# Calculate conservation at each position
alignment = AlignIO.read(aligned_file, 'fasta')
alignment_length = alignment.get_alignment_length()
num_seqs = len(alignment)

# Calculate conservation score for each position
conservation = []
for i in range(alignment_length):
    column = alignment[:, i]
    # Count most common character (excluding gaps)
    chars = [c for c in column if c != '-']
    if chars:
        most_common_count = max([chars.count(c) for c in set(chars)])
        conservation.append(most_common_count / len(chars))
    else:
        conservation.append(0)

# Plot conservation
plt.figure(figsize=(14, 4))
plt.plot(conservation, linewidth=0.5)
plt.xlabel('Alignment Position')
plt.ylabel('Conservation Score')
plt.title('Sequence Conservation Across Alignment')
plt.ylim(0, 1)
plt.axhline(y=0.7, color='r', linestyle='--', alpha=0.3, label='70% conservation')
plt.legend()
plt.tight_layout()
plt.show()

print(f"Mean conservation: {sum(conservation)/len(conservation):.2%}")

## 4. Model Selection with ModelFinder (IQ-TREE)

ModelFinder automatically selects the best-fit substitution model for your data using model selection criteria like BIC, AIC, and AICc.

### Why Model Selection Matters:
- Different evolutionary models make different assumptions about how DNA sequences evolve
- Using an inappropriate model can lead to incorrect tree topology and branch lengths
- ModelFinder tests many models and selects the one that best fits your data

### Common Models:
- **JC69**: Jukes-Cantor (equal rates, equal frequencies)
- **K80**: Kimura 2-parameter (transitions ≠ transversions)
- **HKY**: Hasegawa-Kishino-Yano (K80 + unequal base frequencies)
- **GTR**: General Time Reversible (most parameter-rich)
- **+G**: Gamma rate heterogeneity
- **+I**: Invariant sites

In [ ]:
# Run ModelFinder
print("🔄 Running ModelFinder...\n")

modelfinder_cmd = f"iqtree -s {aligned_file} -m MFP -nt AUTO -pre modelfinder -redo"
result = subprocess.run(modelfinder_cmd, shell=True, capture_output=True, text=True)

if result.returncode == 0:
    print("✓ Model selection completed!\n")
    
    # Parse and display results
    with open('modelfinder.iqtree', 'r') as f:
        content = f.read()
        
        # Extract best model
        if 'Best-fit model:' in content:
            for line in content.split('\n'):
                if 'Best-fit model:' in line:
                    print("📊 " + line.strip())
                    break
        
        # Extract model comparison table
        print("\n📈 Top Models by BIC:")
        if 'BIC' in content:
            in_table = False
            count = 0
            for line in content.split('\n'):
                if 'Model' in line and 'BIC' in line:
                    in_table = True
                    print(line)
                elif in_table and line.strip() and not line.startswith('---'):
                    print(line)
                    count += 1
                    if count >= 5:  # Show top 5 models
                        break
else:
    print(f"❌ Error running ModelFinder: {result.stderr}")

### 💡 Interpretation: Understanding Model Selection

**Questions to consider:**
1. What is the best-fit model selected by ModelFinder?
2. Does it include rate heterogeneity (+G) or invariant sites (+I)?
3. How much better is the best model compared to simpler models?
4. What does this tell you about the evolutionary process in your data?

## 5. Phylogenetic Tree Inference with IQ-TREE

Now we'll construct a maximum likelihood phylogenetic tree using the best-fit model.

### IQ-TREE Features:
- Fast and accurate ML tree inference
- Bootstrap analysis for branch support
- Uses the best model from ModelFinder

### Parameters:
- `-s`: Input alignment file
- `-m`: Evolutionary model (MFP = ModelFinder Plus)
- `-bb`: Ultrafast bootstrap (1000 replicates recommended)
- `-nt`: Number of threads

In [ ]:
# Run IQ-TREE with ultrafast bootstrap
print("🔄 Running IQ-TREE with 1000 ultrafast bootstrap replicates...")
print("   (This may take a few minutes)\n")

tree_prefix = 'phylogeny'
iqtree_cmd = f"iqtree -s {aligned_file} -m MFP -bb 1000 -nt AUTO -pre {tree_prefix} -redo"

result = subprocess.run(iqtree_cmd, shell=True, capture_output=True, text=True)

if result.returncode == 0:
    print("✓ Tree inference completed!\n")
    
    # Display log information
    with open(f'{tree_prefix}.iqtree', 'r') as f:
        content = f.read()
        
        # Extract key statistics
        for line in content.split('\n'):
            if 'Log-likelihood of the tree:' in line:
                print("📊 " + line.strip())
            elif 'Number of free parameters:' in line:
                print("   " + line.strip())
            elif 'AIC score:' in line or 'BIC score:' in line:
                print("   " + line.strip())
    
    print("\n✓ Tree file saved as:", f'{tree_prefix}.treefile')
    print("✓ Full results saved as:", f'{tree_prefix}.iqtree')
else:
    print(f"❌ Error running IQ-TREE: {result.stderr}")

## 6. Tree Visualization with Toytree

Now let's visualize our phylogenetic tree with bootstrap support values.

In [ ]:
# Load the tree
tree_file = f'{tree_prefix}.treefile'
tree = toytree.tree(tree_file)

print(f"📊 Tree Statistics:")
print(f"   Number of tips: {tree.ntips}")
print(f"   Number of nodes: {tree.nnodes}")
print(f"   Tree height: {tree.treenode.height:.6f}")
print(f"   Is rooted: {tree.is_rooted()}")

In [ ]:
# Basic tree visualization
canvas, axes, mark = tree.draw(
    width=1200,
    height=400,
    node_labels='support',
    node_sizes=15,
    node_colors='lightblue',
    tip_labels_align=True,
    tip_labels_style={'font-size': '12px', 'fill': 'white'},
    edge_style={'stroke': 'white', 'stroke-width': 2},
    node_labels_style={'fill': 'white', 'font-size': '11px'},
)

print("\n🌳 Phylogenetic Tree with Bootstrap Support Values")

In [ ]:
# Enhanced tree visualization with color-coded bootstrap support
# Extract support values
support_values = []
for node in tree.treenode.traverse():
    if not node.is_leaf() and node.support is not None:
        support_values.append(float(node.support))

# Define color mapping for bootstrap support
def get_support_color(support):
    if support is None:
        return 'lightgray'
    support = float(support)
    if support >= 95:
        return 'darkgreen'  # Strong support
    elif support >= 70:
        return 'orange'     # Moderate support
    else:
        return 'red'        # Weak support

# Get colors for all nodes
node_colors = []
for node in tree.treenode.traverse('postorder'):
    if node.is_leaf():
        node_colors.append('lightblue')
    else:
        node_colors.append(get_support_color(node.support))

# Plot styled tree with extra width for labels and white colors for dark mode
canvas, axes, mark = tree.draw(
    width=1200,
    height=500,
    node_labels='support',
    node_sizes=18,
    node_colors=node_colors,
    tip_labels_align=True,
    tip_labels_style={'font-size': '14px', 'font-weight': 'bold', 'fill': 'white'},
    edge_widths=2,
    edge_style={'stroke': 'white', 'stroke-width': 2},
    node_labels_style={'fill': 'white', 'font-size': '11px'},
)

print("\n🌳 Styled Phylogenetic Tree")
print("\n🎨 Bootstrap Support Color Legend:")
print("   🟢 Dark Green: ≥95% (Strong support)")
print("   🟠 Orange: 70-94% (Moderate support)")
print("   🔴 Red: <70% (Weak support)")

## 7. Analyzing Branch Lengths

Branch lengths in a phylogenetic tree represent evolutionary distance (usually the number of substitutions per site).

In [ ]:
# Extract branch length information
branch_lengths = []
for node in tree.treenode.traverse():
    if node.dist > 0:  # Exclude root
        branch_lengths.append(node.dist)

# Calculate statistics
import numpy as np
branch_lengths = np.array(branch_lengths)

print("📏 Branch Length Statistics:")
print(f"   Mean: {np.mean(branch_lengths):.6f}")
print(f"   Median: {np.median(branch_lengths):.6f}")
print(f"   Min: {np.min(branch_lengths):.6f}")
print(f"   Max: {np.max(branch_lengths):.6f}")
print(f"   Std Dev: {np.std(branch_lengths):.6f}")

# Plot branch length distribution
plt.figure(figsize=(10, 5))
plt.hist(branch_lengths, bins=20, edgecolor='black', alpha=0.7)
plt.xlabel('Branch Length (substitutions/site)')
plt.ylabel('Frequency')
plt.title('Distribution of Branch Lengths')
plt.axvline(np.mean(branch_lengths), color='red', linestyle='--', 
            label=f'Mean: {np.mean(branch_lengths):.6f}')
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Calculate and visualize pairwise distances
from Bio import Phylo
import numpy as np

# Load tree with BioPython for distance calculations
bio_tree = Phylo.read(tree_file, 'newick')

# Get all tip names
tips = [tip.name for tip in bio_tree.get_terminals()]

# Calculate pairwise distances
n = len(tips)
distance_matrix = np.zeros((n, n))

for i, tip1 in enumerate(tips):
    for j, tip2 in enumerate(tips):
        if i != j:
            distance_matrix[i, j] = bio_tree.distance(tip1, tip2)

# Create heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(distance_matrix, 
            xticklabels=tips, 
            yticklabels=tips,
            cmap='YlOrRd',
            annot=True,
            fmt='.4f',
            cbar_kws={'label': 'Evolutionary Distance'})
plt.title('Pairwise Evolutionary Distances')
plt.tight_layout()
plt.show()

# Find most and least divergent pairs
np.fill_diagonal(distance_matrix, np.inf)
min_idx = np.unravel_index(np.argmin(distance_matrix), distance_matrix.shape)
np.fill_diagonal(distance_matrix, 0)
max_idx = np.unravel_index(np.argmax(distance_matrix), distance_matrix.shape)

print(f"\n🔍 Distance Analysis:")
print(f"   Most similar pair: {tips[min_idx[0]]} ↔ {tips[min_idx[1]]}")
print(f"   Distance: {distance_matrix[min_idx]:.6f}")
print(f"\n   Most divergent pair: {tips[max_idx[0]]} ↔ {tips[max_idx[1]]}")
print(f"   Distance: {distance_matrix[max_idx]:.6f}")

## 8. Interpretation Exercises

### Exercise 1: Bootstrap Support Values

**Questions:**
1. What percentage of your internal branches have bootstrap support ≥95%?
2. Which clades (groups) have the strongest support?
3. Are there any poorly supported branches? What might this indicate?
4. Generally, bootstrap values ≥70% are considered acceptable. How many branches meet this threshold?

In [ ]:
# Analyze bootstrap support distribution
support_values = []
for node in tree.treenode.traverse():
    if not node.is_leaf() and node.support is not None:
        support_values.append(float(node.support))

support_values = np.array(support_values)

print("🎯 Bootstrap Support Analysis:")
print(f"   Total internal branches: {len(support_values)}")
print(f"   Mean support: {np.mean(support_values):.1f}%")
print(f"   Median support: {np.median(support_values):.1f}%")
print(f"\n   Branches with ≥95% support: {np.sum(support_values >= 95)} ({np.sum(support_values >= 95)/len(support_values)*100:.1f}%)")
print(f"   Branches with 70-94% support: {np.sum((support_values >= 70) & (support_values < 95))} ({np.sum((support_values >= 70) & (support_values < 95))/len(support_values)*100:.1f}%)")
print(f"   Branches with <70% support: {np.sum(support_values < 70)} ({np.sum(support_values < 70)/len(support_values)*100:.1f}%)")

# Plot support value distribution
plt.figure(figsize=(10, 5))
plt.hist(support_values, bins=20, edgecolor='black', alpha=0.7, color='steelblue')
plt.xlabel('Bootstrap Support (%)')
plt.ylabel('Number of Branches')
plt.title('Distribution of Bootstrap Support Values')
plt.axvline(95, color='green', linestyle='--', label='Strong support (≥95%)')
plt.axvline(70, color='orange', linestyle='--', label='Moderate support (≥70%)')
plt.legend()
plt.tight_layout()
plt.show()

### Exercise 2: Branch Lengths and Evolutionary Rates

**Questions:**
1. Which terminal branches (leading to tips) are the longest? What does this suggest?
2. Are there any species that appear to be evolving faster than others?
3. What is the total tree length (sum of all branch lengths)?
4. How do internal branch lengths compare to terminal branch lengths?

In [ ]:
# Analyze terminal vs internal branch lengths
terminal_branches = []
internal_branches = []

for node in tree.treenode.traverse():
    if node.dist > 0:  # Exclude root
        if node.is_leaf():
            terminal_branches.append((node.name, node.dist))
        else:
            internal_branches.append(node.dist)

# Sort terminal branches by length
terminal_branches.sort(key=lambda x: x[1], reverse=True)

print("📊 Branch Length Comparison:")
print(f"\n   Terminal branches (tips):")
for name, dist in terminal_branches:
    print(f"      {name}: {dist:.6f}")

print(f"\n   Mean terminal branch length: {np.mean([x[1] for x in terminal_branches]):.6f}")
print(f"   Mean internal branch length: {np.mean(internal_branches):.6f}")

# Calculate total tree length
total_tree_length = sum([x[1] for x in terminal_branches]) + sum(internal_branches)
print(f"\n   Total tree length: {total_tree_length:.6f} substitutions/site")

# Visualize comparison
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.barh([x[0] for x in terminal_branches], [x[1] for x in terminal_branches], 
         color='steelblue', edgecolor='black')
plt.xlabel('Branch Length')
plt.title('Terminal Branch Lengths')
plt.tight_layout()

plt.subplot(1, 2, 2)
data = [internal_branches, [x[1] for x in terminal_branches]]
plt.boxplot(data, labels=['Internal', 'Terminal'])
plt.ylabel('Branch Length')
plt.title('Internal vs Terminal Branches')
plt.tight_layout()
plt.show()

### Exercise 3: Tree Topology

**Questions:**
1. What groups (clades) do you observe in the tree?
2. Do the relationships match what you would expect based on biology/taxonomy?
3. Are there any surprising groupings?
4. Is the tree balanced or imbalanced? What might cause this?

In [ ]:
# Analyze tree balance and structure
from collections import Counter

# Calculate tree balance (Sackin index - lower is more balanced)
def get_depth(node):
    if node.is_leaf():
        return 0
    return 1 + max([get_depth(child) for child in node.children])

def sackin_index(tree_node):
    depths = []
    for leaf in tree_node.iter_leaves():
        depth = 0
        current = leaf
        while current.up:
            depth += 1
            current = current.up
        depths.append(depth)
    return sum(depths)

sackin = sackin_index(tree.treenode)
max_depth = get_depth(tree.treenode)

print("🌳 Tree Structure Analysis:")
print(f"   Maximum depth: {max_depth}")
print(f"   Sackin index: {sackin}")
print(f"   (Lower Sackin index indicates more balanced tree)")

# Identify sister groups
print(f"\n   Sister relationships:")
for node in tree.treenode.traverse():
    if not node.is_leaf() and not node.is_root():
        children = node.children  # Direct attribute access, not get_children()
        if len(children) == 2:
            left_tips = [leaf.name for leaf in children[0].iter_leaves()]
            right_tips = [leaf.name for leaf in children[1].iter_leaves()]
            if len(left_tips) <= 2 and len(right_tips) <= 2:
                print(f"      {left_tips} ↔ {right_tips}")
                if node.support:
                    print(f"      Bootstrap support: {node.support}%")

## 9. Export Results

Let's export our tree and results for further analysis or publication.

In [ ]:
# Save tree in multiple formats
import toyplot.svg

# Save as SVG (vector graphics, good for publications)
# Note: Using dark colors for SVG export (assumes light background for publications)
canvas, axes, mark = tree.draw(
    width=1200,
    height=600,
    node_labels='support',
    node_sizes=18,
    node_colors=node_colors,
    tip_labels_align=True,
    tip_labels_style={'font-size': '14px', 'fill': 'black'},
    edge_widths=2,
    edge_style={'stroke': 'darkgray', 'stroke-width': 2},
    node_labels_style={'fill': 'black', 'font-size': '11px'},
)
toyplot.svg.render(canvas, 'phylogenetic_tree.svg')
print("✓ Tree saved as: phylogenetic_tree.svg")

# Create summary report
summary_report = f"""
PHYLOGENETIC ANALYSIS SUMMARY
============================

Dataset Information:
  - Number of sequences: {len(sequences)}
  - Alignment length: {alignment.get_alignment_length()}
  - Mean conservation: {sum(conservation)/len(conservation):.2%}

Model Selection:
  - Best-fit model: [See modelfinder.iqtree file]

Tree Statistics:
  - Number of tips: {tree.ntips}
  - Total tree length: {total_tree_length:.6f}
  - Mean bootstrap support: {np.mean(support_values):.1f}%
  - Branches with ≥95% support: {np.sum(support_values >= 95)}/{len(support_values)}

Branch Lengths:
  - Mean: {np.mean(branch_lengths):.6f}
  - Range: {np.min(branch_lengths):.6f} - {np.max(branch_lengths):.6f}

Files Generated:
  - Tree file: {tree_prefix}.treefile
  - Alignment: {aligned_file}
  - Full IQ-TREE log: {tree_prefix}.iqtree
  - Tree visualization: phylogenetic_tree.svg
"""

with open('analysis_summary.txt', 'w') as f:
    f.write(summary_report)

print("✓ Summary report saved as: analysis_summary.txt")
print("\n" + summary_report)

### Download Results

Download the following files for your records:
- Tree file (Newick format): For use in other software
- Tree visualization (SVG): For presentations/publications
- Summary report: Overview of analysis

In [ ]:
# Download key files (Colab only)
if IN_COLAB:
    files.download(f'{tree_prefix}.treefile')
    files.download('phylogenetic_tree.svg')
    files.download('analysis_summary.txt')
    print("✓ Files ready for download!")
else:
    print("✓ Files saved locally:")
    print(f"   - {tree_prefix}.treefile")
    print(f"   - phylogenetic_tree.svg")
    print(f"   - analysis_summary.txt")

## 10. Conclusion and Next Steps

Congratulations! You've completed a full phylogenetic analysis workflow.

### What You've Learned:
1. How to align sequences with MAFFT
2. How to select appropriate evolutionary models
3. How to infer maximum likelihood trees with bootstrap support
4. How to interpret bootstrap values and branch lengths
5. How to visualize and analyze phylogenetic trees

### Further Exploration:
- Try different alignment methods (Clustal Omega, MUSCLE)
- Experiment with different tree inference methods (RAxML, MrBayes for Bayesian inference)
- Root your tree with an outgroup
- Estimate divergence times with molecular clocks
- Test alternative tree topologies
- Perform ancestral state reconstruction

### Resources:
- [IQ-TREE Documentation](http://www.iqtree.org/doc/)
- [MAFFT Manual](https://mafft.cbrc.jp/alignment/software/manual/manual.html)
- [Toytree Documentation](https://toytree.readthedocs.io/)
- [Evomics Workshops](https://evomics.org)

---

**Questions or feedback?** Visit [learn.evomics.org](https://learn.evomics.org)

**Citation:** If you use this tutorial, please cite:
- IQ-TREE: Nguyen et al. (2015) Mol Biol Evol 32:268-274
- MAFFT: Katoh & Standley (2013) Mol Biol Evol 30:772-780
- Toytree: Eaton (2020) Methods Ecol Evol 11:187-191
"